In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder, OneHotEncoder
from xverse.transformer import WOE
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Task 3: Feature Engineering

# 1. Load the dataset
from sklearn.impute import KNNImputer


file_path = "../data/credit_scoring_data.csv"
df = pd.read_csv(file_path)

# 2. Check the FraudResult column
print("Unique values in FraudResult:", df["FraudResult"].unique())
print("Data type of FraudResult:", df["FraudResult"].dtype)

# Ensure FraudResult is binary
df["FraudResult"] = df["FraudResult"].astype(int)

# 3. Create aggregate features
aggregate_features = df.groupby("CustomerId").agg(
    Total_Transaction_Amount=("Amount", "sum"),
    Average_Transaction_Amount=("Amount", "mean"),
    Transaction_Count=("TransactionId", "count"),
    Std_Transaction_Amount=("Amount", "std")
).reset_index()
df = df.merge(aggregate_features, on="CustomerId", how="left")

# 4. Extract temporal features
df["TransactionStartTime"] = pd.to_datetime(df["TransactionStartTime"])
df["Transaction_Hour"] = df["TransactionStartTime"].dt.hour
df["Transaction_Day"] = df["TransactionStartTime"].dt.day
df["Transaction_Month"] = df["TransactionStartTime"].dt.month
df["Transaction_Year"] = df["TransactionStartTime"].dt.year

# 5. Encode categorical variables
one_hot_cols = ["CurrencyCode", "CountryCode", "ChannelId", "PricingStrategy"]
df = pd.get_dummies(df, columns=one_hot_cols, drop_first=True)

label_cols = ["ProviderId", "ProductId", "ProductCategory"]
for col in label_cols:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])

# 6. Handle missing values
numerical_cols = df.select_dtypes(include=["float64", "int64"]).columns
imputer = KNNImputer(n_neighbors=5)
df[numerical_cols] = imputer.fit_transform(df[numerical_cols])

# 7. Normalize/Standardize numerical features
scaler = StandardScaler()
df[numerical_cols] = scaler.fit_transform(df[numerical_cols])

# 8. Feature Engineering using WoE and IV (using optbinning)
from optbinning import OptimalBinning

# Define the target variable
target = "FraudResult"

# Ensure the target variable is binary
df[target] = df[target].astype(int)

# Perform WoE binning for a numerical feature (e.g., Total_Transaction_Amount)
optb = OptimalBinning(name="Total_Transaction_Amount", dtype="numerical", solver="cp")
optb.fit(df["Total_Transaction_Amount"], df[target])
df["Total_Transaction_Amount_WoE"] = optb.transform(df["Total_Transaction_Amount"], metric="woe")

# Display the binning table
print(optb.binning_table.build())

# 9. Construct a default estimator (proxy)
df["Risk_Label"] = np.where(df["Total_Transaction_Amount"] > df["Total_Transaction_Amount"].median(), 1, 0)

# 10. Save the processed dataset
processed_file_path = "../data/processed_credit_scoring_data.csv"
df.to_csv(processed_file_path, index=False)
print(f"Processed dataset saved to {processed_file_path}")

Unique values in FraudResult: [0 1]
Data type of FraudResult: int64


/home/azazh/Documents/10-acadamy/w-6/CreditScoring/envcredit/lib/python3.10/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/home/azazh/Documents/10-acadamy/w-6/CreditScoring/envcredit/lib/python3.10/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/home/azazh/Documents/10-acadamy/w-6/CreditScoring/envcredit/lib/python3.10/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/home/azazh/Documents/10-acadamy/w-6/CreditScoring/envcredit/lib/python3.10/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/home/azazh/Document

                 Bin  Count  Count (%)  Non-event  Event  Event rate  \
0       (-inf, 0.19)  65905   0.688936      65900      5    0.000076   
1       [0.19, 0.23)  16920   0.176873      16905     15    0.000887   
2       [0.23, 0.28)   6719   0.070237       6690     29    0.004316   
3        [0.28, inf)   6118   0.063954       5974    144    0.023537   
4            Special      0   0.000000          0      0    0.000000   
5            Missing      0   0.000000          0      0    0.000000   
Totals                95662   1.000000      95469    193    0.002018   

             WoE        IV        JS  
0       3.282589  2.180853  0.192496  
1       0.823448  0.081812  0.009947  
2      -0.762793  0.061164  0.007465  
3      -2.478508  1.694156  0.170159  
4            0.0  0.000000  0.000000  
5            0.0  0.000000  0.000000  
Totals            4.017985  0.380068  
Processed dataset saved to ../data/processed_credit_scoring_data.csv


In [3]:
# # Aggregate features for each customer
# agg_features = df.groupby("CustomerId").agg(
#     total_transaction_amount=("Amount", "sum"),
#     avg_transaction_amount=("Amount", "mean"),
#     transaction_count=("Amount", "count"),
#     std_transaction_amount=("Amount", "std")
# ).reset_index()

# # Merge back with the original DataFrame if necessary
# df = df.merge(agg_features, on="CustomerId", how="left")

# print(agg_features.head())


# Extract Features


In [4]:
# # Ensure TransactionStartTime is in datetime format
# df["TransactionStartTime"] = pd.to_datetime(df["TransactionStartTime"])

# # Extract features
# df["transaction_hour"] = df["TransactionStartTime"].dt.hour
# df["transaction_day"] = df["TransactionStartTime"].dt.day
# df["transaction_month"] = df["TransactionStartTime"].dt.month
# df["transaction_year"] = df["TransactionStartTime"].dt.year

# print(df[["TransactionStartTime", "transaction_hour", "transaction_day", "transaction_month", "transaction_year"]].head())


# Encode Categorical Variables


In [5]:
# # One-Hot Encoding
# df = pd.get_dummies(df, columns=["ProductCategory", "ChannelId", "PricingStrategy"], drop_first=True)

# # For demonstration: Label Encoding (if required)
# from sklearn.preprocessing import LabelEncoder

# le = LabelEncoder()
# df["CurrencyCode_encoded"] = le.fit_transform(df["CurrencyCode"])



In [6]:
# from datetime import datetime
# import pandas as pd

# # Ensure TransactionStartTime is in datetime format and timezone-naive
# df["TransactionStartTime"] = pd.to_datetime(df["TransactionStartTime"]).dt.tz_localize(None)

# # Recency: Days since the last transaction
# current_date = pd.Timestamp.now(tz=None)  # Ensure current_date is timezone-naive
# df["transaction_date"] = df["TransactionStartTime"]  # Keep it as a datetime object
# last_transaction = df.groupby("CustomerId")["transaction_date"].max().reset_index()

# # Calculate recency in days (difference between current date and last transaction date)
# last_transaction["recency"] = (current_date - last_transaction["transaction_date"]).dt.days

# # Frequency, Monetary, Size
# rfms = df.groupby("CustomerId").agg(
#     frequency=("TransactionId", "count"),  # Count of transactions
#     monetary=("Amount", "sum"),           # Sum of transaction amounts
#     size=("Amount", "mean")               # Average transaction amount
# ).reset_index()

# # Merge recency with RFMS data
# rfms = rfms.merge(last_transaction[["CustomerId", "recency"]], on="CustomerId", how="left")

# # Display the result
# print(rfms.head())


In [7]:
# import seaborn as sns
# import matplotlib.pyplot as plt

# # Visualize RFMS
# sns.pairplot(rfms, diag_kind="kde")
# plt.show()

# # Define thresholds for high/low RFMS scores (adjust thresholds based on visualization)
# rfms["rfms_score"] = rfms.apply(lambda x: "good" if (x["recency"] < 30 and x["frequency"] > 5 and x["monetary"] > 1000) else "bad", axis=1)
